In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import dblquad

In [3]:
Kxs = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1, 2, 0, -2, 0,
                2, 1, -1, -2, -2, -1, 1, 2, 2, -2, -2, 2,
                3, 0, -3, 0, 3, 1, -1, -3, -3, -1, 1, 3,
                3, 2, -2, -3, -3, -2, 2, 3, 4, 0, -4, 0,
                4, 1, -1, -4, -4, -1, 1, 4, 3, -3, -3, 3,
                4, 2, -2, -4, -4, -2, 2, 4])
Kys = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1, 0, 2, 0, -2,
                1, 2, 2, 1, -1, -2, -2, -1, 2, 2, -2, -2,
                0, 3, 0, -3, 1, 3, 3, 1, -1, -3, -3, -1,
                2, 3, 3, 2, -2, -3, -3, -2, 0, 4, 0, -4,
                1, 4, 4, 1, -1, -4, -4, -1, 3, 3, -3, -3,
                2, 4, 4, 2, -2, -4, -4, -2])

Ne = 69
Ns = 138
t = 1j

Lx = np.sqrt(2*np.pi*Ns/np.imag(t))
Ly = Lx*np.imag(t)

radii_unscaled = np.arange(0.4, 0.501, 0.01)
radii = radii_unscaled*Ly
state = "free_fermions"

In [4]:
def InnerProductReal(y,x,n,m):
    return np.cos( (2*np.pi/Lx) * ((Kxs[n]-Kxs[m])*x + (Kys[n]-Kys[m])*y) ) / (Lx*Ly)
def InnerProductImag(y,x,n,m):
    return np.sin( (2*np.pi/Lx) * ((Kxs[n]-Kxs[m])*x + (Kys[n]-Kys[m])*y) ) / (Lx*Ly)

In [5]:
for i in range(radii.size):
    print(f"Circle radius {radii[i]}")
    M = np.zeros((Ne, Ne), dtype=np.complex128)
    real_errors = np.zeros((Ne, Ne), dtype=np.float64)
    imag_errors = np.zeros((Ne, Ne), dtype=np.float64)

    for n in range(Ne):
        print(f"Calculating diagonal overlap of state {n}..")
        real, real_err = dblquad(InnerProductReal, Lx/2-radii[i], Lx/2+radii[i],
                                 lambda x: (
            Ly/2 - np.sqrt(radii[i]**2 - (x-Lx/2)**2)),
            lambda x: (
            Ly/2 + np.sqrt(radii[i]**2 - (x-Lx/2)**2)),
            args=(n, n))
        M[n, n] = real
        real_errors[n, n] = real_err

    for n in range(Ne):
        for m in range(n+1, Ne):
            print(f"Calculating overlap of states {n,m}..")
            real, real_err = dblquad(InnerProductReal, Lx/2-radii[i], Lx/2+radii[i],
                                     lambda x: (
                                     Ly/2 - np.sqrt(radii[i]**2 - (x-Lx/2)**2)),
                                     lambda x: (
                                     Ly/2 + np.sqrt(radii[i]**2 - (x-Lx/2)**2)),
                                     args=(n, m))
            imag, imag_err = dblquad(InnerProductImag, Lx/2-radii[i], Lx/2+radii[i],
                                     lambda x: (
                                     Ly/2 - np.sqrt(radii[i]**2 - (x-Lx/2)**2)),
                                     lambda x: (
                                     Ly/2 + np.sqrt(radii[i]**2 - (x-Lx/2)**2)),
                                     args=(n, m))
            M[n, m] = (real+1j*imag)
            real_errors[n, m] = real_err
            imag_errors[n, m] = imag_err

            
            M[m, n] = (real-1j*imag)
            real_errors[m, n] = real_err
            imag_errors[m, n] = imag_err

    np.save(f"{state}_overlaps_N_{Ne}_circle_{radii_unscaled[i]:.3f}.npy", M)
    np.save(
        f"{state}_overlaps_real_err_N_{Ne}_circle_{radii_unscaled[i]:.3f}.npy", real_errors)
    np.save(
        f"{state}_overlaps_imag_err_N_{Ne}_circle_{radii_unscaled[i]:.3f}.npy", imag_errors)

Circle radius 11.778485963082236
Calculating diagonal overlap of state 0..
Calculating diagonal overlap of state 1..
Calculating diagonal overlap of state 2..
Calculating diagonal overlap of state 3..
Calculating diagonal overlap of state 4..
Calculating diagonal overlap of state 5..
Calculating diagonal overlap of state 6..
Calculating diagonal overlap of state 7..
Calculating diagonal overlap of state 8..
Calculating diagonal overlap of state 9..
Calculating diagonal overlap of state 10..
Calculating diagonal overlap of state 11..
Calculating diagonal overlap of state 12..
Calculating diagonal overlap of state 13..
Calculating diagonal overlap of state 14..
Calculating diagonal overlap of state 15..
Calculating diagonal overlap of state 16..
Calculating diagonal overlap of state 17..
Calculating diagonal overlap of state 18..
Calculating diagonal overlap of state 19..
Calculating diagonal overlap of state 20..
Calculating diagonal overlap of state 21..
Calculating diagonal overlap of